In [399]:
import pandas as pd
import numpy as np
import pandasql as ps
from pandasql import sqldf

**Load csv datasets**

In [402]:
Users= pd.read_csv ('USER_TAKEHOME (1).csv')
Transactions = pd.read_csv ('TRANSACTION_TAKEHOME (1).csv')
Products = pd.read_csv('PRODUCTS_TAKEHOME (1).csv')
Products

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11
...,...,...,...,...,...,...,...
845547,Health & Wellness,Topical Muscle & Joint Relief Treatments,Braces & Wraps,NaN,NaN,NaN,7.223016e+11
845548,Snacks,Cookies,NaN,NaN,"TREEHOUSE FOODS, INC.",LOFTHOUSE,4.182082e+10
845549,Snacks,Candy,Confection Candy,NaN,HARIBO GMBH & CO KG,HARIBO,1.001672e+11
845550,Snacks,Nuts & Seeds,Hazelnuts,NaN,DOUBLE-COLA CO,JUMBO,7.539076e+10


### What are the top 5 brands by receipts scanned among users 21 and over?


In [405]:


# SQL query using pandasql
query = """
With UserAge as 
(
Select ID, BIRTH_DATE,
CASE WHEN CAST(strftime('%m', DATE('now')) AS INTEGER) > CAST(strftime('%m', BIRTH_DATE) AS INTEGER)
             OR (CAST(strftime('%m', DATE('now')) AS INTEGER) = CAST(strftime('%m', BIRTH_DATE) AS INTEGER) 
             AND CAST(strftime('%d', DATE('now')) AS INTEGER) >= CAST(strftime('%d', BIRTH_DATE) AS INTEGER)) 
        THEN CAST(strftime('%Y', DATE('now')) AS INTEGER) - CAST(strftime('%Y', BIRTH_DATE) AS INTEGER)
        ELSE CAST(strftime('%Y', DATE('now')) AS INTEGER) - CAST(strftime('%Y', BIRTH_DATE) AS INTEGER) - 1
    END AS Age
From Users
)

, Brand_Count
as
(
Select distinct BRAND, count(*) as total_count
From(
    Select  ID, Age, t.BARCODE, p.BARCODE, BRAND
    From (Select distinct RECEIPT_ID, BARCODE, USER_ID FROM Transactions) t 
         INNER JOIN (Select ID, AGE From UserAge Where Age >= 21) cte on t.USER_ID = cte.ID
         INNER JOIN (Select distinct * From Products) p on p.BARCODE = t.BARCODE
    )
Group By BRAND
)
, Brand_Rank as
(
Select BRAND, total_count, dense_rank() over (Order By total_count DESC) as Brand_Dense_Rank
        , row_number() over (Order By total_count DESC) as Brand_Number_Rank
From Brand_Count
Where Brand is not null 
)

SELECT a.BRAND, a.Total_Count, a.Brand_Dense_Rank
FROM Brand_Rank a
JOIN (SELECT * FROM Brand_Rank WHERE Brand_Number_Rank = 5) b
    ON a.Brand_Number_Rank <= 5 
    OR a.Brand_Dense_Rank = b.Brand_Dense_Rank;

"""

# Run the query
result = sqldf(query)
print(result)

             BRAND  total_count  Brand_Dense_Rank
0             DOVE            3                 1
1      NERDS CANDY            3                 1
2        COCA-COLA            2                 2
3      GREAT VALUE            2                 2
4        HERSHEY'S            2                 2
5           MEIJER            2                 2
6  SOUR PATCH KIDS            2                 2
7          TRIDENT            2                 2


**Answer:**  

The top brands by receipts scanned among users aged 21 and over are as follows:
DOVE, NERDS CANDY, COCA-COLA, GREAT VALUE, HERSHEY'S, MEIJER, SOUR PATCH KIDS, TRIDENT

Since there is a tie for the second position, I recommend considering the top 8 brands to ensure all tied brands are highlighted. To ensure the data is standardized by receipt and avoid counting products from the same brand multiple times, I am using the query:

SELECT DISTINCT RECEIPT_ID, BARCODE, USER_ID FROM Transactions;

It's important to note that the analysis is based on transactions from known users in the Users table. However, there are transactions that couldn't be matched to a user, which means the total number of transactions may be higher than what is captured in this analysis.

### What are the top 5 brands by sales among users that have had their account for at least six months?

In [526]:
query2 = """

With months_over_6 as
(
SELECT ID, CREATED_DATE, DATE('now') AS today
FROM Users
WHERE 
    (CAST(strftime('%Y', 'now') AS INTEGER) - CAST(strftime('%Y', CREATED_DATE) AS INTEGER)) * 12 
    + (CAST(strftime('%m', 'now') AS INTEGER) - CAST(strftime('%m', CREATED_DATE) AS INTEGER)) >= 6
)
,
Brand_Totalsale as
(
Select Distinct BRAND, SUM(Total_Sale) over (Partition By BRAND ) as Brand_TotalSale
From (
Select Distinct ID, CREATED_DATE, RECEIPT_ID, FINAL_QUANTITY*FINAL_SALE as Total_Sale, BRAND
From months_over_6 INNER JOIN Transactions t on months_over_6.ID = t.USER_ID
     INNER JOIN Products p on p.BARCODE = t.BARCODE)
Where BRAND is not null
)

Select BRAND, '$' || Brand_TotalSale, Brand_Rank
From (
    Select BRAND,  Brand_TotalSale , Dense_Rank () Over (Order By Brand_TotalSale DESC) as Brand_Rank 
    From Brand_Totalsale
    )
Where Brand_Rank <=5 

"""


# Run the query
result2 = sqldf(query2)
print(result2)

         BRAND '$' || Brand_TotalSale  Brand_Rank
0          CVS                  $72.0           1
1         DOVE                 $30.91           2
2     TRESEMMÉ                 $29.16           3
3      TRIDENT                 $23.36           4
4  COORS LIGHT                 $17.48           5


**Answer:**  
The top brands by sales among users who have had their accounts for at least six months are as follows:
1. CVS
2. DOVE
3. TRESEMMÉ
4. TRIDENT
5. COORS LIGHT  
To calculate the total sales, I used the formula **final_quantity * final_sale**, which gives the total sales value for each brand. Using only the final_quantity would give the number of items purchased, while using only final_sale would reflect the sale price per item. This approach ensures we are analyzing both the volume and value of sales to identify the top brands by total sales.

Please note that this result is based on transactions from known users, and there may be additional transactions that could not be matched with users in the Users table.

### What is the percentage of sales in the Health & Wellness category by generation?

**Generation:**

The Greatest Generation: Born 1901–1924  
The Silent Generation: Born 1925–1945  
The Baby Boomer Generation: Born 1946–1964  
Generation X: Born 1965–1979  
Millennials: Born 1981–1996  
Generation Z: Born 1997–2012  
Gen Alpha: Born 2013–2025

Data Source: Wikipedia https://en.wikipedia.org/wiki/Generation#:~:text=A%20generation%20is%20all%20of,biogenesis%2C%20reproduction%2C%20and%20procreation.

In [501]:
query3 = """
With User_Gen as
(
SELECT ID, BIRTH_DATE,
    CASE 
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1901' AND '1924' THEN 'The Greatest Generation'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1925' AND '1945' THEN 'The Silent Generation'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1946' AND '1964' THEN 'Baby Boomers'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1965' AND '1980' THEN 'Generation X'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1981' AND '1996' THEN 'Millennials'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1997' AND '2012' THEN 'Generation Z'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '2013' AND '2025' THEN 'Gen Alpha'
        Else 'Unknown'
    END AS Generation
FROM Users
)
, Total as 
(
Select CATEGORY_1, sum(Total_Sale) as 'Total_HealthWellness_Sale'
From (Select USER_ID, RECEIPT_ID, BARCODE, FINAL_QUANTITY*FINAL_SALE as Total_Sale From Transactions) t 
     INNER JOIN (Select distinct * From Products Where CATEGORY_1 ='Health & Wellness' ) p on t.BARCODE = p.BARCODE
Group By CATEGORY_1
)

Select Distinct Generation ,  '$' ||sum(Total_Sale) over (Partition By Generation) as Gen_Sum,  '$' ||Total_HealthWellness_Sale
      , ROUND((100 * SUM(Total_Sale) OVER (PARTITION BY Generation)) / Total_HealthWellness_Sale, 2) || '%'  as Percentage
From User_Gen INNER JOIN (Select USER_ID, RECEIPT_ID, BARCODE, FINAL_QUANTITY*FINAL_SALE as Total_Sale From Transactions) t on User_Gen.ID = t.USER_ID
     INNER JOIN (Select distinct * From Products Where CATEGORY_1 ='Health & Wellness' ) p on t.BARCODE = p.BARCODE ,Total

 

"""

# Run the query
result3 = sqldf(query3)
print(result3)

     Generation Gen_Sum '$' ||Total_HealthWellness_Sale Percentage
0  Baby Boomers  $86.56                     $29549.2988      0.29%
1  Generation X  $52.39                     $29549.2988      0.18%
2   Millennials  $35.17                     $29549.2988      0.12%


**Answer:**  
The percentage of sales in the Health & Wellness category by generation is as follows:  
Baby Boomers: 0.29%  
Generation X: 0.18%  
Millennials: 0.12%  

The percentage is calculated by dividing the total sales for each generation (Gen_Sum) by the overall total sales in the Health & Wellness category (Total_HealthWellness_Sale), which includes both known and unknown user transactions, and then multiplying by 100.

I have chosen to include both known and unknown users' transactions in the total sales calculation. This approach ensures that the analysis is more adaptable, so if we later access a dataset containing full user information, the query can be run seamlessly without needing further adjustments.

### Who are Fetch’s power users?

Direction:  
1. Users by Generation (based on Match ID for User-Transaction): Identify how many different generations are represented in Fetch’s power users by analyzing the Match ID for each User-Transaction.
2. Gender / Language
3. Category Interests: Determine which product categories are more popular.

**After experimenting with various approaches, I concluded that focusing on Category Interests provides the most valuable insights for targeting and marketing efforts.**

**1. Most User of Generation based on Match ID for User - Transaction**

In [416]:
query4 = """
With User_Gen as
(
SELECT ID, BIRTH_DATE,
    CASE 
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1901' AND '1924' THEN 'The Greatest Generation'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1925' AND '1945' THEN 'The Silent Generation'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1946' AND '1964' THEN 'Baby Boomers'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1965' AND '1980' THEN 'Generation X'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1981' AND '1996' THEN 'Millennials'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '1997' AND '2012' THEN 'Generation Z'
        WHEN strftime('%Y', BIRTH_DATE) BETWEEN '2013' AND '2025' THEN 'Gen Alpha'
        Else 'Unknown'
    END AS Generation
FROM Users
)

, Distinct_Receipt as
(Select distinct USER_ID, RECEIPT_ID 
From User_Gen INNER JOIN Transactions t on User_Gen.ID = t.USER_ID
)

Select distinct Generation, 
        (SELECT COUNT(DISTINCT RECEIPT_ID) FROM Distinct_Receipt) as 'Total distinct receipt_Id of known users ', 
        Count(DISTINCT RECEIPT_ID) as total_Gen_Count, 
        Round(100.0* Count(DISTINCT RECEIPT_ID)/(SELECT COUNT(DISTINCT RECEIPT_ID) FROM Distinct_Receipt),4) || '%' as Percentage
From User_Gen INNER JOIN Distinct_Receipt dr on User_Gen.ID = dr.USER_ID
Group By Generation
Order By total_Gen_Count DESC
 

"""

# Run the query
result4 = sqldf(query4)
print(result4)

              Generation  Total distinct receipt_Id of known users   \
0           Generation X                                        125   
1            Millennials                                        125   
2           Baby Boomers                                        125   
3  The Silent Generation                                        125   
4                Unknown                                        125   
5           Generation Z                                        125   

   total_Gen_Count Percentage  
0               45      36.0%  
1               39      31.2%  
2               36      28.8%  
3                3       2.4%  
4                1       0.8%  
5                1       0.8%  


**2. Analyze through User Gender and Language**

In [450]:
query4 = """
With User_info as
(
SELECT ID, GENDER, LANGUAGE
FROM Users
)

, Distinct_Receipt as
(Select distinct USER_ID, RECEIPT_ID 
From User_info INNER JOIN Transactions t on User_info.ID = t.USER_ID
)

Select distinct GENDER, LANGUAGE, 
        (SELECT COUNT(DISTINCT RECEIPT_ID) FROM Distinct_Receipt) as 'Total distinct receipt_Id of known users ', 
        Count(DISTINCT RECEIPT_ID) as total_Gend_Lan_Count, 
        Round(100.0* Count(DISTINCT RECEIPT_ID)/(SELECT COUNT(DISTINCT RECEIPT_ID) FROM Distinct_Receipt),4) || '%' as Percentage
From User_info INNER JOIN Distinct_Receipt dr on User_info.ID = dr.USER_ID
Group By GENDER, LANGUAGE
Order By total_Gend_Lan_Count DESC
 

"""

# Run the query
result4 = sqldf(query4)
print(result4)

   GENDER LANGUAGE  Total distinct receipt_Id of known users   \
0  female       en                                        125   
1    male       en                                        125   
2  female   es-419                                        125   
3    None       en                                        125   

   total_Gend_Lan_Count Percentage  
0                    98      78.4%  
1                    21      16.8%  
2                     5       4.0%  
3                     1       0.8%  


**3. Analyze by Category to determine user preference based on Receipt_ID.**

In [462]:
# Get the Top3 Category_1
query4_1 = """
Select CATEGORY_1 , Count(distinct RECEIPT_ID)as Cat1_Count
From (Select distinct RECEIPT_ID, USER_ID, BARCODE From Transactions) t 
     INNER JOIN Products p on t.BARCODE = p.BARCODE
Group By CATEGORY_1
Order By Cat1_Count DESC
Limit 3
 

"""

# Get the SubCategory of the top3 Category_1
result4_1 = sqldf(query4_1)
print(result4_1)

query4_2 = """

With cte as (
Select CATEGORY_1 , Count(distinct RECEIPT_ID)as Cat1_Count_All_Receipt
From (Select distinct RECEIPT_ID, USER_ID, BARCODE From Transactions) t 
     INNER JOIN Products p on t.BARCODE = p.BARCODE
Group By CATEGORY_1
Order By Cat1_Count_All_Receipt DESC
Limit 3
)
,

cte2 as (
Select CATEGORY_1 , CATEGORY_2, Count(distinct RECEIPT_ID)as Cat12_Count
From (Select distinct RECEIPT_ID, USER_ID, BARCODE From Transactions) t 
     INNER JOIN Products p on t.BARCODE = p.BARCODE
     LEFT JOIN Users on Users.ID = t.USER_ID
Group By CATEGORY_1,CATEGORY_2--, GENDER
)
,
cte2_Rank as (
Select cte2.CATEGORY_1 , CATEGORY_2,  Cat12_Count, 
       Round(100.0 *Cat12_Count / Cat1_Count_All_Receipt,2) ||'%' as 'Percentage of Cat1',
       dense_rank() Over (Partition By cte2.CATEGORY_1  Order By Cat12_Count  DESC) as rank
From cte2 INNER JOIN cte on cte2.CATEGORY_1 = cte.CATEGORY_1
)

Select * 
From cte2_Rank
Where rank <=3
 

"""

# Run the query
result4_2 = sqldf(query4_2)
print(result4_2)

          CATEGORY_1  Cat1_Count
0             Snacks        7463
1  Health & Wellness        3415
2          Beverages         812
          CATEGORY_1                CATEGORY_2  Cat12_Count  \
0          Beverages    Carbonated Soft Drinks          753   
1          Beverages                     Water           56   
2          Beverages  Fruit & Vegetable Juices            3   
3  Health & Wellness    Medicines & Treatments          738   
4  Health & Wellness               Bath & Body          505   
5  Health & Wellness                 Hair Care          473   
6             Snacks                     Candy         2049   
7             Snacks                     Chips         1343   
8             Snacks               Snack Cakes          636   

  Percentage of Cat1  rank  
0             92.73%     1  
1               6.9%     2  
2              0.37%     3  
3             21.61%     1  
4             14.79%     2  
5             13.85%     3  
6             27.46%     1  
7    

**Answer:**  
To identify Fetch's power users, I analyzed transaction data from June to September 2024, focusing on their category preferences. Based on this analysis:

Snacks emerged as the most popular category, leading with 7,463 transactions. Within Snacks, Candy is the top subcategory, making up 27.46% of all Snack transactions, followed by Chips at 18% and Snack Cakes at 8.52%.

Health & Wellness ranked second in popularity, with 3,415 transactions. The subcategories within this group are led by Medicines & Treatments (21.61%), followed by Bath & Body (14.79%) and Hair Care (13.85%).

Beverages showed a preference for Carbonated Soft Drinks, which dominated the category with 92.73% of all Beverage transactions, significantly outpacing Water (6.9%) and Fruit & Vegetable Juices (0.37%).

This analysis highlights the strong preferences of Fetch's most active users, with a clear inclination towards Snacks, Health & Wellness, and Beverages, particularly Carbonated Soft Drinks. This information can guide targeted marketing strategies for Fetch’s power users.


### Which is the leading brand in the Dips & Salsa category?
To determine the leading brand, I am using the following metrics:
- Brand Count: The number of occurrences for each brand. This indicates popularity or market presence.

- Brand Sale Sum: Total sales revenue. This measures the monetary contribution of each brand.

- Brand Sale Percentage: The percentage of total sales that each brand represents..=

In [384]:
query5 = """

With totalsum as (
Select CATEGORY_2,  sum(Total_Sale)as All_Dip_Sum
From (Select distinct RECEIPT_ID, USER_ID, BARCODE, FINAL_QUANTITY*FINAL_SALE as Total_Sale From Transactions) t 
     INNER JOIN (Select distinct * From Products) p on t.BARCODE = p.BARCODE
Where CATEGORY_2 = 'Dips & Salsa' and BRAND is not null
)
, cte as
(
Select BRAND, Count(distinct RECEIPT_ID)as Brand_Count,
       sum(Total_Sale)as Brand_Sale_Sum, Round(100.0* sum(Total_Sale) / All_Dip_Sum,2) || '%' as Brand_Sale_Percentage
From (Select distinct RECEIPT_ID, USER_ID, BARCODE, FINAL_QUANTITY*FINAL_SALE as Total_Sale From Transactions) t 
     INNER JOIN (Select distinct * From Products) p on t.BARCODE = p.BARCODE
     ,totalsum
Where p.CATEGORY_2 = 'Dips & Salsa' 
Group By BRAND
)

Select BRAND, Brand_Count, Dense_Rank () Over (Order By Brand_Count DESC) as Brand_Count_Rank,
       Brand_Sale_Sum, Brand_Sale_Percentage, Dense_Rank () Over (Order By Brand_Sale_Percentage DESC) as Brand_Sale_Sum_Rank
From cte
Order By Brand_Count_Rank, Brand_Sale_Sum_Rank 


"""

# Run the query
result5 = sqldf(query5)
print(result5)

                    BRAND  Brand_Count  Brand_Count_Rank  Brand_Sale_Sum  \
0                TOSTITOS           36                 1          197.24   
1                    PACE           24                 2           85.75   
2                    None           21                 3          107.21   
3                  FRITOS           19                 4           73.76   
4        DEAN'S DAIRY DIP           17                 5           39.95   
5              MARKETSIDE           16                 6           65.22   
6            HELUVA GOOD!           15                 7           53.98   
7    FRESHNESS GUARANTEED           12                 8           46.66   
8                MARZETTI           11                 9           51.14   
9           HIDDEN VALLEY           10                10           76.38   
10             GOOD FOODS            9                11           94.91   
11    HIDDEN VALLEY RANCH            9                11           41.82   
12          

**Answer:**  
Based on the analysis of transaction purchase data from June to September, **TOSTITOS** leads the category with the highest Brand_Count (36) and the highest Brand_Sale_Sum ($197.24), accounting for 14.46% of total sales. It holds the #1 position in both popularity and revenue.

### At what percent has Fetch grown year over year?
**For calculating the growth rate, I decided to use the metric: New User Growth by Year – Account Creation.**

This approach focuses on the number of new users who created accounts each year, providing a straightforward way to measure Fetch’s growth in user acquisition.

In [376]:
query6 = """

With year_count as
(
Select strftime('%Y', CREATED_DATE) as Year , Count(distinct ID) as New_Users
From Users
Group by strftime('%Y', CREATED_DATE) 
)

Select a.Year, a.New_Users, 
       Case When b.New_Users is not NUll Then Round(100.0 *(a.New_Users - b.New_Users)/ b.New_Users ,2) ||'%'
       Else 'N/A' END as growth_rate_percentage
From year_count a LEFT JOIN year_count b on  CAST(a.Year AS INTEGER)  = CAST(b.Year AS INTEGER)+1

"""

# Run the query
result6 = sqldf(query6)
print(result6)

    Year  New_Users growth_rate_percentage
0   2014         30                    N/A
1   2015         51                  70.0%
2   2016         70                 37.25%
3   2017        644                 820.0%
4   2018       2168                236.65%
5   2019       7093                227.17%
6   2020      16883                138.02%
7   2021      19159                 13.48%
8   2022      26807                 39.92%
9   2023      15464                -42.31%
10  2024      11631                -24.79%


**Answer:**  
I defined new users as those who create an account on the platform, ignoring active time for now.
  
For this question, I am focusing on the year-over-year growth percentage of new users for Fetch Rewards. Fetch experienced substantial growth in its early years, particularly from 2017 to 2020, with growth rates as high as 820% in 2017 and 138% in 2020. The peak in new users occurred in 2022, with a total of 26,807 new users. However, the growth rate turned negative in both 2023 and 2024, which may indicate changes in the environment. This decline could be influenced by factors such as increased competition, substitutes, market shifts, or changes in customer behavior.

To better understand the reasons behind this decline and guide future strategies, I plan to conduct a deeper analysis of the demographics and behaviors of new users. This will include examining how they interact with the platform over time and assessing potential seasonal trends or external factors that may have influenced growth during these periods. By exploring these areas, Fetch can develop more targeted marketing strategies to enhance user acquisition and retention in the future.